# b_bottom & potential field

In [ ]:
from setproctitle import setproctitle
setproctitle("prepare")

import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "3"

In [ ]:
import glob 
import numpy as np

from cmspinn.prepare_hmi import PrepareHMI
from cmspinn.potential_field import potential_cube

In [ ]:
Nz = 160
batch_size = 2500

hmi_path = '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/'
base_path = os.path.expanduser('~/workspace/workspace_mine/_data/NOAA12673/')

b_bottom_path = os.path.join(base_path, 'b_bottom')
b_potential_path = os.path.join(base_path, 'b_potential')

os.makedirs(b_bottom_path, exist_ok=True)
os.makedirs(b_potential_path, exist_ok=True)

In [ ]:
creater = PrepareHMI(b_bottom_path, hmi_path)
creater.load_hmi_data(idx1=247, idx2=248)

[('/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_083600_TAI.Bp.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_083600_TAI.Bt.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_083600_TAI.Br.fits')]

In [ ]:
creater.save_bottom(stride=2)

Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_083600.npy


See https://docs.sunpy.org/en/stable/code_ref/map.html#fixing-map-metadata for how to fix metadata before loading it with sunpy.map.Map.
See https://fits.gsfc.nasa.gov/fits_standard.html forthe FITS unit standards. [sunpy.map.mapbase]


In [ ]:
b_bottom_list = sorted(glob.glob(os.path.join(b_bottom_path, '*.npy')))
b_bottom_list

['/userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_083600.npy']

In [ ]:
for b_bottom_path in b_bottom_list:
    b_bottom_date = os.path.basename(b_bottom_path)[9:-4]
    b_pot_vtk_path = os.path.join(b_potential_path, f'b_potential_{b_bottom_date}.vtk')

    if not os.path.exists(b_pot_vtk_path):
        with open(b_bottom_path, 'rb') as f:
            b_bottom = np.load(f)

        bz_bottom = b_bottom[:, :, 2]
        potential_maker = potential_cube(bz_bottom, Nz)

        potential_maker.cal_and_save_potential_field(b_pot_vtk_path, batch_size=batch_size)

        print(f'Saved {b_pot_vtk_path}')

    else:
        print(f'Exist {b_pot_vtk_path}')

Potential Field: 100%|██████████| 4932/4932 [03:04<00:00, 26.80it/s]


Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_potential/b_potential_20170906_083600.vtk
